In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score,root_mean_squared_error
import joblib

pd.set_option('display.max_columns', None)

In [10]:
df_train=pd.read_csv('../../data/train_cleaned_data.csv')
df_test=pd.read_csv('../../data/test_cleaned_data.csv')
df_train.head(5)

,Day_Count,Day,StoreID,ProductName,specials,Amt,Inflation_Percentage,Unemployment_Percentage
0,1,Tuesday,S001,Veg Burger,0,13125.0,4.7,1.7
1,1,Tuesday,S001,Fries,0,11250.0,4.7,1.7
2,1,Tuesday,S001,Coca-Cola,0,17680.0,4.7,1.7
3,1,Tuesday,S002,Cheese Burger,0,7800.0,4.7,1.7
4,1,Tuesday,S002,Fries,0,16275.0,4.7,1.7


In [11]:
df_train[df_train['Day_Count']==100]

,Day_Count,Day,StoreID,ProductName,specials,Amt,Inflation_Percentage,Unemployment_Percentage
1568,100,Wednesday,S002,Fries,0,12075.00,3.0,1.9
1569,100,Wednesday,S001,Cheese Burger,0,9200.00,3.0,1.9
1570,100,Wednesday,S003,Cheese Burger,0,6600.00,3.0,1.9
1571,100,Wednesday,S001,Falafel Burger,1,8370.00,3.0,1.9
1572,100,Wednesday,S001,Chicken Burger,0,13430.00,3.0,1.9
1573,100,Wednesday,S003,Coca-Cola,0,21845.00,3.0,1.9
1574,100,Wednesday,S003,Falafel Burger,1,8370.00,3.0,1.9
1575,100,Wednesday,S003,Fries,0,11625.00,3.0,1.9
1576,100,Wednesday,S002,Chicken Burger,0,7990.00,3.0,1.9
1577,100,Wednesday,S003,Chicken Burger,0,8500.00,3.0,1.9


In [12]:

def aggregate_date(df):
    product_daily_sales = df.groupby(['Day_Count', 'ProductName']).agg({
        'Day': 'first',
        'specials': 'sum',  
        'Inflation_Percentage': 'first',
        'Unemployment_Percentage': 'first',
        'Amt': 'sum' 
    }).reset_index()

    
    product_daily_sales['Weekend'] = product_daily_sales['Day'].isin(['Friday','Saturday', 'Sunday']).astype(int)
    
    product_daily_sales = pd.get_dummies(product_daily_sales, columns=['ProductName', 'Day'])
    return product_daily_sales

train_product_daily_sales=aggregate_date(df_train)
test_product_daily_sales=aggregate_date(df_test)
train_product_daily_sales.head(10)


,Day_Count,specials,Inflation_Percentage,Unemployment_Percentage,Amt,Weekend,ProductName_Cheese Burger,ProductName_Chicken Burger,ProductName_Coca-Cola,ProductName_Custom,ProductName_Falafel Burger,ProductName_Fries,ProductName_Veg Burger,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday
0,1,0,4.7,1.7,28600.00,0,True,False,False,False,False,False,False,False,False,False,False,False,True,False
1,1,0,4.7,1.7,35530.00,0,False,True,False,False,False,False,False,False,False,False,False,False,True,False
2,1,0,4.7,1.7,69785.00,0,False,False,True,False,False,False,False,False,False,False,False,False,True,False
3,1,0,4.7,1.7,41850.00,0,False,False,False,False,False,True,False,False,False,False,False,False,True,False
4,1,0,4.7,1.7,65175.15,0,False,False,False,False,False,False,True,False,False,False,False,False,True,False
5,2,0,4.7,1.7,21200.00,0,True,False,False,False,False,False,False,False,False,False,False,False,False,True
6,2,0,4.7,1.7,24990.00,0,False,True,False,False,False,False,False,False,False,False,False,False,False,True
7,2,0,4.7,1.7,49215.00,0,False,False,True,False,False,False,False,False,False,False,False,False,False,True
8,2,0,4.7,1.7,0.00,0,False,False,False,True,False,False,False,False,False,False,False,False,False,True
9,2,0,4.7,1.7,30600.00,0,False,False,False,False,False,True,False,False,False,False,False,False,False,True


In [13]:
train_product_daily_sales['Store_Count'].unique()

KeyError: 'Store_Count'

In [ ]:
train_product_daily_sales[train_product_daily_sales['Store_Count']>3]

,Day_Count,Store_Count,specials,Inflation_Percentage,Unemployment_Percentage,Amt,Weekend,ProductName_Cheese Burger,ProductName_Chicken Burger,ProductName_Coca-Cola,ProductName_Custom,ProductName_Falafel Burger,ProductName_Fries,ProductName_Veg Burger,Day_Friday,Day_Monday,Day_Saturday,Day_Sunday,Day_Thursday,Day_Tuesday,Day_Wednesday
544,100,4,0,3.0,1.9,80410.00,0,False,False,True,False,False,False,False,False,False,False,False,False,False,True
550,101,4,0,3.0,1.9,82535.00,0,False,False,True,False,False,False,False,False,False,False,False,True,False,False
556,102,4,0,3.0,1.9,103785.00,1,False,False,True,False,False,False,False,True,False,False,False,False,False,False
563,103,4,0,3.0,1.9,100640.00,1,False,False,True,False,False,False,False,False,False,True,False,False,False,False
570,104,4,0,3.0,1.9,100895.00,1,False,False,True,False,False,False,False,False,False,False,True,False,False,False
1573,280,4,0,2.2,3.2,0.00,0,False,False,False,True,False,False,False,False,True,False,False,False,False,False
2828,500,4,0,4.8,1.1,71675.15,0,False,False,False,False,False,False,True,False,False,False,False,True,False,False


In [ ]:
# avg_product_sales = df.groupby('ProductName')['Amt'].mean().to_dict()
# product_daily_sales['avg_product_price'] = product_daily_sales['ProductName'].map(avg_product_sales)

In [ ]:
feature_cols = [col for col in train_product_daily_sales.columns if col not in ['Day_Count', 'Amt']]
x_train = train_product_daily_sales[feature_cols]
y_train = train_product_daily_sales['Amt']

x_test = test_product_daily_sales[feature_cols]
y_test = test_product_daily_sales['Amt']


In [ ]:

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    min_samples_leaf=3,
    random_state=42
)

rf_model.fit(x_train, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,5
,min_samples_leaf,3
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [ ]:
y_pred = rf_model.predict(x_test)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Performance:")
print(f"MAE: ${mae:.2f}")
print(f"R2 Score: {r2:.3f}")

Model Performance:
MAE: $1640.22
R2 Score: 0.990


In [ ]:
# model would have learned that zero sales are coming from custom category of productname
custom_mask = y_test == 0  
non_zero_mae = mean_absolute_error(y_test[~custom_mask], y_pred[~custom_mask])
non_zero_r2 = r2_score(y_test[~custom_mask], y_pred[~custom_mask])

print(f"Performance excluding zero sales:") 
print(f"MAE: ${non_zero_mae:,.2f}")
print(f"R2: {non_zero_r2:.3f}")
print(f"Zero sales: {custom_mask.sum()} out of {len(y_test)}")

Performance excluding zero sales:
MAE: $1,788.93
R2: 0.985
Zero sales: 103 out of 1239


In [ ]:
rmse = root_mean_squared_error(y_test, y_pred)
print(f"RMSE: ${rmse:,.2f}")
print(f"RMSE/MAE ratio: {rmse/1788.93:.2f}") 

RMSE: $2,315.82
RMSE/MAE ratio: 1.29


The ratio is under 1.5 so it is acceptable.

In [ ]:
# joblib.dump(rf_model, 'rf_model.pkl')

['rf_model.pkl']